*** Step 1: 📦 Install Required Python Packages***

In [ ]:
!pip install flask flask-ngrok pyngrok streamlit colab_everything


In [ ]:
!pip install flask flask-ngrok pyngrok


In [ ]:
!pip install colab-everything pyngrok streamlit


In [ ]:
!pip install pyngrok


In [ ]:
!pip install flask pyngrok


In [ ]:
!pip install streamlit pyngrok


*** Step 2: 🔐 Set Up ngrok Authentication***



In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("303pUHw4QeaPPT8ti0FBCo6sKhp_6R7CyNNKFPNK1h17M2yZk")

*** Step 3: 📂 Upload Dataset File (Interactively)***

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving ds_salaries.csv to ds_salaries (1).csv


*** Step 4: 🔧 Import Libraries and Create Flask App***

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import pandas as pd
import threading

df = pd.read_csv("ds_salaries.csv")
app = Flask(__name__)

@app.route('/')
def home():
    return "✅ Flask backend is running"

@app.route('/predict', methods=['POST'])
def predict():
    if df is None:
        return jsonify({'error': 'Dataset not loaded'}), 500
    data = request.get_json()
    prediction = df['salary_in_usd'].mean()
    return jsonify({'predicted_salary': round(prediction, 2)})

# 🔄 Start Flask on a new port
def run_flask():
    app.run(port=5002)

threading.Thread(target=run_flask).start()

# 🌍 Create ngrok tunnel on same port
public_url = ngrok.connect(5002)
print("🔗 Public ngrok URL:", public_url)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5002
INFO:werkzeug:Press CTRL+C to quit


🔗 Public ngrok URL: NgrokTunnel: "https://b77528a5b59a.ngrok-free.app" -> "http://localhost:5002"


*** Step 5: 📺 (Commented Out) Streamlit App Code***


In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import requests
import time
import plotly.graph_objects as go
import os

st.set_page_config(page_title="NextGen Salary Dashboard", page_icon="💼", layout="wide")

# 🎨 Styling + Slogan
st.markdown("""
<style>
    html, body, [class*="css"] {
        background-color: #f0f4f8;
    }
    h1 {
        color: #FF4B4B;
        font-size: 48px;
        text-align: center;
    }
    .stButton button {
        background-color: #FF4B4B;
        color: white;
        font-size: 18px;
        border-radius: 8px;
    }
</style>
""", unsafe_allow_html=True)

st.markdown("<h1>🚀 Unlock Your True Earning Potential</h1>", unsafe_allow_html=True)
st.markdown("""
    <h4 style='text-align: center; color: #2C3E50; font-style: italic;'>
    AI-powered insights to elevate your career journey.
    </h4>
""", unsafe_allow_html=True)
st.markdown("---")

# 🔍 Form
with st.form("predict_form"):
    col1, col2 = st.columns(2)
    with col1:
        experience = st.selectbox("Experience Level", ["Entry", "Mid", "Senior", "Executive"])
        employment = st.selectbox("Employment Type", ["Full-time", "Part-time", "Contract", "Internship"])
        job_title = st.text_input("Job Title")
        residence = st.text_input("Employee Residence")
    with col2:
        remote_ratio = st.slider("Remote Ratio (%)", 0, 100, 50)
        company_location = st.text_input("Company Location")
        company_size = st.selectbox("Company Size", ["Small", "Medium", "Large"])
        resume = st.file_uploader("📄 Upload Resume (PDF/CSV)", type=["pdf", "csv"])

    submit = st.form_submit_button("🔍 Predict & Analyze")

# 💡 Output Section
if submit:
    with st.spinner("🎯 Predicting... and analyzing your profile"):
        time.sleep(2)
        payload = {
            "experience_level": experience,
            "employment_type": employment,
            "job_title": job_title,
            "employee_residence": residence,
            "remote_ratio": remote_ratio,
            "company_location": company_location,
            "company_size": company_size
        }

        salary = None
        try:
            response = requests.post("http://127.0.0.1:5000/predict", json=payload)
            salary = response.json().get('predicted_salary')
        except Exception:
            st.warning("🔄 Prediction service is offline. Showing estimated insights.")

        if salary:
            st.success(f"💰 Predicted Salary: ${salary}")
            st.balloons()
        else:
            salary = 70000
            st.info(f"💰 Estimated Salary: ${salary} (default)")

        # 📈 Salary Growth Chart
        years = list(range(1, 6))
        projected = [round(salary * (1.05 ** y)) for y in years]
        fig = go.Figure(data=go.Scatter(x=years, y=projected, mode='lines+markers'))
        fig.update_layout(title="📈 Salary Growth Over 5 Years", xaxis_title="Years", yaxis_title="USD")
        st.plotly_chart(fig)

        # 🧠 Career Suggestions
        st.markdown("### 🧠 Career Suggestions Based on Your Profile")
        st.info("You're a strong match for: **ML Engineer**, **Data Analyst**, **BI Developer**")

        # 🌍 Country Comparison Chart
        st.markdown("### 🌍 Top 5 Country Comparison")
        countries = ["US 🇺🇸", "Germany 🇩🇪", "India 🇮🇳", "UK 🇬🇧", "Canada 🇨🇦"]
        salaries = [salary * 1.2, salary * 1.15, salary * 0.8, salary, salary * 1.05]
        country_fig = go.Figure([go.Bar(x=countries, y=salaries, marker_color='indianred')])
        country_fig.update_layout(title="🌍 Top 5 Country Comparison")
        st.plotly_chart(country_fig)

        # 📄 Resume Feedback (Placeholder)
        if resume:
            st.markdown("### 📝 Resume Analysis & Skill Gaps")
            st.warning("Resume parsed: Looking for keywords... 🚧 *(Feature in development)*")

        # 📥 PDF Export
        st.markdown("### 📄 Export Your Insights")
        try:
            with open("report.pdf", "rb") as f:
                pdf_bytes = f.read()
            st.download_button("📥 Export PDF Report", data=pdf_bytes, file_name="salary_report.pdf", mime="application/pdf")
        except FileNotFoundError:
            st.warning("📄 PDF file not found. Please add 'report.pdf' to your folder.")


Overwriting streamlit_app.py


**Step 6: ▶️ Run Streamlit App**

In [ ]:
!nohup streamlit run streamlit_app.py --server.port 8501 &



nohup: appending output to 'nohup.out'


**Step 7: 🌐 Create Public ngrok URL for Flask App**

In [ ]:
public_url = ngrok.connect(8501)
print("🔗 Public URL:", public_url)



🔗 Public URL: NgrokTunnel: "https://f94d4dea3472.ngrok-free.app" -> "http://localhost:8501"
